In [1]:
import numpy as np
import pandas as pd
import os
from itertools import product
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf

from statsmodels.tsa.stattools import adfuller

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


dataset=pd.read_csv("/Users/alishadaredia/Desktop/Ryerson - Arif/CIND820/Price dataset/BaseMetalPricePrediction.csv")
dataset.describe()
dataset.head()

,Date,Price,Open,High,Low,Vol.,Change %,Commodity,US CPI,China CPI,US PMI,China PMI
0,2023-08-31,"25,396.00","25,300.00","25,150.00","25,110.00",1.25K,-0.31%,Tin,140.801768,103.2,47.6,49.7
1,2023-08-30,"25,475.00","25,400.00","25,300.00","25,325.00",1.27K,0.32%,Tin,140.801768,103.2,47.6,49.7
2,2023-08-29,"25,395.00","25,200.00","25,050.00","25,055.00",1.32K,-0.40%,Tin,140.801768,103.2,47.6,49.7
3,2023-08-25,"25,497.00","25,600.00","25,500.00","25,575.00",0.59K,-1.44%,Tin,140.801768,103.2,47.6,49.7
4,2023-08-24,"25,870.00","25,900.00","25,825.00","25,850.00",0.39K,-0.98%,Tin,140.801768,103.2,47.6,49.7


In [2]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14928 entries, 0 to 14927
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14928 non-null  object 
 1   Price      14928 non-null  object 
 2   Open       14928 non-null  object 
 3   High       14928 non-null  object 
 4   Low        14928 non-null  object 
 5   Vol.       7661 non-null   object 
 6   Change %   14928 non-null  object 
 7   Commodity  14928 non-null  object 
 8   US CPI     14928 non-null  float64
 9   China CPI  14928 non-null  float64
 10  US PMI     14928 non-null  float64
 11  China PMI  14928 non-null  float64
dtypes: float64(4), object(8)
memory usage: 1.4+ MB


In [3]:
dataset["Date"] = dataset["Date"].astype('datetime64[ns]')
dataset["Price"] = dataset["Price"].str.replace(",", "")
dataset["Open"] = dataset["Open"].str.replace(",", "")
dataset["High"] = dataset["High"].str.replace(",", "")
dataset["Low"] = dataset["Low"].str.replace(",", "")

dataset["Price"] = dataset["Price"].astype('float64')
dataset["Open"] = dataset["Open"].astype('float64')
dataset["High"] = dataset["High"].astype('float64')
dataset["Low"] = dataset["Low"].astype('float64')
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14928 entries, 0 to 14927
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       14928 non-null  datetime64[ns]
 1   Price      14928 non-null  float64       
 2   Open       14928 non-null  float64       
 3   High       14928 non-null  float64       
 4   Low        14928 non-null  float64       
 5   Vol.       7661 non-null   object        
 6   Change %   14928 non-null  object        
 7   Commodity  14928 non-null  object        
 8   US CPI     14928 non-null  float64       
 9   China CPI  14928 non-null  float64       
 10  US PMI     14928 non-null  float64       
 11  China PMI  14928 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 1.4+ MB


In [4]:
dataset.describe()

,Price,Open,High,Low,US CPI,China CPI,US PMI,China PMI
count,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000
mean,11675.353999,11675.556693,11738.159450,11609.762911,112.731793,90.784734,53.440287,50.652693
std,8620.380610,8627.157952,8649.762122,8593.869623,10.922922,8.582517,5.496316,2.249782
min,848.000000,848.000000,848.000000,848.000000,96.410318,74.960150,32.400000,35.700000
25%,2868.000000,2868.000000,2868.000000,2868.000000,105.392893,84.520710,50.600000,50.000000
50%,9520.000000,9525.250000,9555.500000,9499.000000,109.724818,91.083430,53.600000,50.600000
75%,18405.500000,18394.000000,18510.625000,18301.250000,117.847979,98.972540,57.300000,51.500000
max,81051.500000,81066.500000,86791.500000,81031.500000,140.801768,104.000000,64.700000,56.600000


In [5]:
for col in ["Commodity"]:
    print(f"Unique values for {col}: {dataset[col].unique()}")

Unique values for Commodity: ['Tin' 'Nickel' 'Lead' 'Copper']


In [6]:
import matplotlib
matplotlib.use('TkAgg')
sns.heatmap(dataset.corr(numeric_only=True))
plt.show()

In [7]:
import pandas_profiling

# Generate profile report
profile = pandas_profiling.ProfileReport(dataset)

# Output report to HTML file
profile.to_file("MetalPrice_EDAoutput.html")

/Users/alishadaredia/opt/anaconda3/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/alishadaredia/opt/anaconda3/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
/var/folders/bk/6666c1x143v01stjr104px_r0000gn/T/ipykernel_65734/100127969.py:1: DeprecationWarning: `import 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
dataset.head()

,Date,Price,Open,High,Low,Vol.,Change %,Commodity,US CPI,China CPI,US PMI,China PMI
0,2023-08-31,25396.0,25300.0,25150.0,25110.0,1.25K,-0.31%,Tin,140.801768,103.2,47.6,49.7
1,2023-08-30,25475.0,25400.0,25300.0,25325.0,1.27K,0.32%,Tin,140.801768,103.2,47.6,49.7
2,2023-08-29,25395.0,25200.0,25050.0,25055.0,1.32K,-0.40%,Tin,140.801768,103.2,47.6,49.7
3,2023-08-25,25497.0,25600.0,25500.0,25575.0,0.59K,-1.44%,Tin,140.801768,103.2,47.6,49.7
4,2023-08-24,25870.0,25900.0,25825.0,25850.0,0.39K,-0.98%,Tin,140.801768,103.2,47.6,49.7


In [7]:
for name, group in dataset.groupby('Commodity'):
    plt.hist(group['Open'], label=name)
    
plt.show()

In [8]:
# Create a figure and axis
fig, ax = plt.subplots()

# Iterate through groups and plot histograms
for name, group in dataset.groupby('Commodity'):
    ax.hist(group['Price'], label=name, alpha=0.5)  

# Add axis titles and labels
ax.set_xlabel('Close Price')
ax.set_ylabel('Frequency')

# Add a legend
ax.legend()

plt.show()


In [9]:
#Check the duplicates
dataset.duplicated().sum()

0

In [10]:
#Check the null values

dataset.isnull().sum()

Date            0
Price           0
Open            0
High            0
Low             0
Vol.         7267
Change %        0
Commodity       0
US CPI          0
China CPI       0
US PMI          0
China PMI       0
dtype: int64

In [11]:
plt.figure(figsize=(15,10))
sns.heatmap(dataset.corr(numeric_only=True), annot = True)

plt.show()

In [20]:
# Convert 'Date' column to datetime and set it as the index
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', inplace=True)

# Resample to a monthly frequency, considering only values on the 1st of every month
monthly_cpi = dataset['US CPI'].resample('MS').mean()

# Create a line plot for the trend
plt.figure(figsize=(10, 6))
plt.plot(monthly_cpi.index, monthly_cpi, marker='o', linestyle='-')
plt.title('US CPI Trend (Monthly)')
plt.xlabel('Date')
plt.ylabel('US CPI')
plt.grid(True)

plt.show()

In [ ]:
# Convert 'Date' column to datetime and set it as the index
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', inplace=True)

# Resample to a monthly frequency, considering only values on the 1st of every month
monthly_cpi = dataset['China CPI'].resample('MS').mean()

# Create a line plot for the trend
plt.figure(figsize=(10, 6))
plt.plot(monthly_cpi.index, monthly_cpi, marker='o', linestyle='-')
plt.title('China CPI Trend (Monthly)')
plt.xlabel('Date')
plt.ylabel('China CPI')
plt.grid(True)

plt.show()